## Vehicle Detection Project

The goals / steps of this project are the following:

- Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
- Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector. 
- Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
- Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
- Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
- Estimate a bounding box for vehicles detected.


### Steps to take:

##### Train classifier
1. load dataset
2. extract features from dataset - play with different feature parameters and combinations
3. scale features using StandardScaler()
4. split data into train and test set
5. train classifier and test accuracy - linear SVC and other - Important is only the time to predict not the time to train since this happens only once. Maybe do RandomizedSearchCV or GridSearchCV

##### Build pipeline for video processing
6. whole image HOG (or sliding window)
7. extract same exact features as before from every search window
8. scale features using same scaler
9. predict using classifier (and print found windows onto test images)
10. add heat to heatmap
11. integrate over several video frames and threshold
12. use label() to get boxes to draw and draw


In [23]:
# Import statements
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
import pandas as pd

#%matplotlib qt
%matplotlib inline

In [24]:
# Load dataset
images = glob.glob('./dataset/vehicles/*/*.png')
cars = []
for image in images:
    cars.append(image)

notcars = []
images = glob.glob('./dataset/non-vehicles/*/*.png')
for image in images:
    notcars.append(image)

print(len(cars), len(notcars))

8792 8968


In [25]:
from skimage.feature import hog


# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block,
                     vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient,
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  transform_sqrt=True,
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:
        features = hog(img, orientations=orient,
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       transform_sqrt=True,
                       visualise=vis, feature_vector=feature_vec)
        return features


# Define a function to compute binned color features
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel()
    # Return the feature vector
    return features


# Define a function to compute color histogram features
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:, :, 0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:, :, 1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:, :, 2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features


# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                     hist_bins=32, orient=9,
                     pix_per_cell=8, cell_per_block=2, hog_channel=0,
                     spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        # apply color conversion if other than 'RGB'
        if color_space != 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else:
            feature_image = np.copy(image)

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
            # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:, :, channel],
                                                         orient, pix_per_cell, cell_per_block,
                                                         vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)
            else:
                hog_features = get_hog_features(feature_image[:, :, hog_channel], orient,
                                                pix_per_cell, cell_per_block, vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features


In [26]:
# extract features from dataset

### TODO: Tweak these parameters and see how the results change.
color_space = 'YCrCb'  # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8  # HOG pixels per cell
cell_per_block = 2  # HOG cells per block
hog_channel = 'ALL'  # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16)  # Spatial binning dimensions
hist_bins = 16  # Number of histogram bins
spatial_feat = True  # Spatial features on or off
hist_feat = True  # Histogram features on or off
hog_feat = True  # HOG features on or off
y_start_stop = [360, 720]  # Min and max in y to search in slide_window()

t = time.time()
car_features = extract_features(cars, color_space=color_space,
                                spatial_size=spatial_size, hist_bins=hist_bins,
                                orient=orient, pix_per_cell=pix_per_cell,
                                cell_per_block=cell_per_block,
                                hog_channel=hog_channel, spatial_feat=spatial_feat,
                                hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space,
                                   spatial_size=spatial_size, hist_bins=hist_bins,
                                   orient=orient, pix_per_cell=pix_per_cell,
                                   cell_per_block=cell_per_block,
                                   hog_channel=hog_channel, spatial_feat=spatial_feat,
                                   hist_feat=hist_feat, hog_feat=hog_feat)

t2 = time.time()
print(round(t2 - t, 2), 'Seconds to extract features...')

C:\Users\qi10487\AppData\Local\Continuum\Anaconda3\envs\carnd-term1\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


222.88 Seconds to extract features...


In [27]:
X = np.vstack((car_features, notcar_features)).astype(np.float64)
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:', orient, 'orientations', pix_per_cell,
      'pixels per cell and', cell_per_block, 'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC
svc = LinearSVC()
# Check the training time for the SVC
t = time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2 - t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample

Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 6108
32.59 Seconds to train SVC...
Test Accuracy of SVC =  0.9859


In [ ]:
extraction_features = {'color_space' : 'YCrCb',
'orient' : 9, 
'pix_per_cell' : 8, 
'cell_per_block' : 2,  
'hog_channel' : 'ALL',  
'spatial_size' : (16, 16),  
'hist_bins' : 16,  
'spatial_feat' : True,  
'hist_feat' : True,  
'hog_feat' : True,  
'y_start_stop' : [360, 720]}

pd.Series(extraction_features)

In [ ]:
print(extraction_features['spatial_size'])

In [72]:
extraction_parameters_pd = pd.read_table("extraction_parameters.csv", sep=';')

print(extraction_parameters_pd.shape)
extraction_parameters_pd['accuracy'] = 0.993

extraction_parameters_pd.to_csv("extraction_parameters.csv", sep=';', index=False)

extraction_parameters_pd



(4, 13)


,orient,pix_per_cell,cell_per_block,hog_channel,spatial_size,hist_bins,spatial_feat,hist_feat,hog_feat,y_start_stop,accuracy,time_to_train,time_to_predict
0,9,8,2,ALL,"(16, 16)",16,True,True,True,"[360, 720]",0.993,0,0
1,12,8,2,0,"(8, 8)",16,True,True,True,"[360, 720]",0.993,0,0
2,9,8,2,1,"(16, 16)",16,True,True,True,"[360, 720]",0.993,0,0
3,9,8,2,ALL,"(16, 16)",16,True,True,True,"[360, 720]",0.993,0,0


# Video

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    return pipeline_vehicle_detection(image, debug=False)

In [ ]:
project_output = 'project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
left = Line('left')
right = Line('right')
clip1 = VideoFileClip("project_video.mp4")
project_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time project_clip.write_videofile(project_output, audio=False)

# Challenge Video

In [ ]:
project_output = 'challenge_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
left = Line('left')
right = Line('right')
clip1 = VideoFileClip("challenge_video.mp4")
project_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time project_clip.write_videofile(project_output, audio=False)

# Harder Challenge Video

In [ ]:
project_output = 'harder_challenge_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
left = Line('left')
right = Line('right')
clip1 = VideoFileClip("harder_challenge_video.mp4")
project_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time project_clip.write_videofile(project_output, audio=False)

# Backup
---

In [ ]:
def sobel_parameters(img, 
                     thresh_abs_x = (0.01, 1), 
                     thresh_abs_y = (0.01, 1), 
                     thresh_mag = (0.05, 1), 
                     thresh_dir = (0, 1)):
    
    # vertical contrast borders. Note that a Scharr kernel is used here for better performance 
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=-1)
    
    # horizontal contrast borders. Scharr kernel
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=-1)
    
    # diagonal contrast borders high->low. Scharr kernel
    kernel10 = np.array([[0, -3, -10],[3, 0, -3],[10, 3, 0]]) 
    sobel10 = cv2.filter2D(img, -1, kernel10)
    
    # diagonal contrast borders low->high. Scharr kernel
    kernel01 = np.array([[10, 3, 0],[3, 0, -3],[0, -3, -10]]) 
    sobel01 = cv2.filter2D(img, -1, kernel01)
     
    # calculate value (sharpness) of contrast border and apply threshold to it
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    
    # threshold values 
    scaled_sobelx = abs_sobelx/np.max(abs_sobelx)
    binary_output_x = np.zeros_like(scaled_sobelx)
    binary_output_x[(scaled_sobelx >= thresh_abs_x[0]) & (scaled_sobelx <= thresh_abs_x[1])] = 1
    
    scaled_sobely = abs_sobely/np.max(abs_sobely)
    binary_output_y = np.zeros_like(scaled_sobely)
    binary_output_y[(scaled_sobely >= thresh_abs_y[0]) & (scaled_sobely <= thresh_abs_y[1])] = 1
    
    mag = np.sqrt(sobelx**2 + sobely**2)
    scaled_mag = mag/np.max(mag)
    binary_output_mag = np.zeros_like(scaled_mag)
    binary_output_mag[(scaled_mag >= thresh_mag[0]) & (scaled_mag <= thresh_mag[1])] = 1
    
    dir_grad = np.arctan2(abs_sobely, abs_sobelx)
    binary_output_dir = np.zeros_like(dir_grad)
    binary_output_dir[(dir_grad > thresh_dir[0]) & (dir_grad <= thresh_dir[1])] = 1
    
    # put all binary images into one big numpy array
    bin_out = img.copy()
    bin_out = np.dstack((bin_out, binary_output_x))
    bin_out = np.dstack((bin_out, binary_output_y))
    bin_out = np.dstack((bin_out, binary_output_mag))
    bin_out = np.dstack((bin_out, binary_output_dir))
    
    dir_mag = np.zeros_like(binary_output_dir)
    dir_mag[(bin_out[:,:,3] == 1) & (bin_out[:,:,4] == 1)] = 1
    bin_out = np.dstack((bin_out, dir_mag))
    
    combined = np.zeros_like(binary_output_dir)
    combined[(bin_out[:,:,1] == 1) & (bin_out[:,:,2] == 1) & (bin_out[:,:,3] == 1) & (bin_out[:,:,4] == 1)] = 1
    bin_out = np.dstack((bin_out, combined))
    
    
    return bin_out

In [ ]:
img = mpimg.imread('./test_images/test5.jpg')
plt.figure(figsize=(17,5))
plt.imshow(img)

bin_out = sobel_parameters(img)
plt.figure(figsize=(17,10))
plt.imshow(bin_out[:,:,1:4], cmap='gray')


plt.figure(figsize=(17,10))
plt.imshow(bin_out[:,:,1], cmap='gray')
plt.figure(figsize=(17,10))
plt.imshow(bin_out[:,:,2], cmap='gray')
plt.figure(figsize=(17,10))
plt.imshow(bin_out[:,:,3], cmap='gray')

In [ ]:
from IPython.html.widgets import *


def sobel(thresh_abs_x_low = 0.1, thresh_abs_x_high = 1,
         thresh_abs_y_low = 0, thresh_abs_y_high = 1, 
         thresh_mag_low = 0, thresh_mag_high = 1,
         thresh_dir_low = 0, thresh_dir_high=0.67):
    
    
    img = mpimg.imread('./test_images/test1.jpg')
    gray = get_redchannel(img)
    birdview = birdview_warp(gray)
    bin_out = sobel_parameters(birdview, 
                               thresh_abs_x = (thresh_abs_x_low, thresh_abs_x_high), 
                                thresh_abs_y = (thresh_abs_y_low, thresh_abs_y_high), 
                                thresh_mag = (thresh_mag_low, thresh_mag_high), 
                                thresh_dir = (thresh_dir_low, thresh_dir_high))
    
    plt.figure(figsize=(17,10))
    #out = np.dstack((bin_out[:,:,1], bin_out[:,:,2], bin_out[:,:,5]))
    plt.imshow(bin_out[:,:,6], cmap='gray')
    plt.show()
    return

interact(sobel, thresh_abs_x_low = (0,1,0.01), thresh_abs_x_high = (0,1,0.01),
         thresh_abs_y_low = (0,1,0.01), thresh_abs_y_high = (0,1,0.01), 
         thresh_mag_low = (0,1,0.01), thresh_mag_high = (0,1,0.01),
         thresh_dir_low = (0,1.57,0.01), thresh_dir_high= (0,1.57,0.01))

In [ ]:
img = mpimg.imread('./test_images/test1.jpg')
print_example_images(img, birdview_warp(img))    

In [ ]:
## sobel demo
img = mpimg.imread('./test_images/circle.png')
plt.figure(figsize=(17,5))
plt.imshow(img)

gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=-1)
sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=-1)
print(sobelx.shape, sobelx.max(), sobelx.min())
placeholder = gray.copy()

# set parameters
thresh_abs_x = (150, 255)
thresh_abs_y = (150, 255)
thresh_mag = (150, 255)
thresh_dir = (0, 0.1)

# abs
abs_sobelx = np.absolute(sobelx)
scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
binary_output_x = np.zeros_like(scaled_sobelx)
binary_output_x[(scaled_sobelx > thresh_abs_x[0]) & (scaled_sobelx < thresh_abs_x[1])] = 1
placeholder = np.dstack((placeholder, binary_output_x))

abs_sobely = np.absolute(sobely)
scaled_sobely = np.uint8(255*abs_sobely/np.max(abs_sobely))
binary_output_y = np.zeros_like(scaled_sobely)
binary_output_y[(scaled_sobely > thresh_abs_y[0]) & (scaled_sobely < thresh_abs_y[1])] = 1
placeholder = np.dstack((placeholder, binary_output_y))

# mag
mag = np.sqrt(sobelx**2 + sobely**2)
scaled_mag = np.uint8(255*mag/np.max(mag))
binary_output_mag = np.zeros_like(scaled_mag)
binary_output_mag[(scaled_mag > thresh_mag[0]) & (scaled_mag < thresh_mag[1])] = 1
placeholder = np.dstack((placeholder, binary_output_mag))

# direction
#thresh_dir = (thresh_dir*np.pi)-(np.pi/2)
dir_grad = np.arctan2(abs_sobely, abs_sobelx)
binary_output_dir = np.zeros_like(dir_grad)
binary_output_dir[(dir_grad > thresh_dir[0]) & (dir_grad <= thresh_dir[1])] = 1
placeholder = np.dstack((placeholder, binary_output_dir))


plt.figure(figsize=(17,5))
plt.imshow(gray, cmap='gray')

plt.figure(figsize=(17,5))
plt.imshow(binary_output_x, cmap='gray')


plt.figure(figsize=(17,5))
plt.imshow(binary_output_y, cmap='gray')

plt.figure(figsize=(17,5))
plt.imshow(binary_output_mag, cmap='gray')

plt.figure(figsize=(17,5))
plt.imshow(binary_output_dir, cmap='gray')

print(placeholder.shape)

In [ ]:
abs_sobelx = np.absolute(sobelx)
print(abs_sobelx.min(), abs_sobelx.max())
scaled_sobelx = abs_sobelx/np.max(abs_sobelx)
print(scaled_sobelx.min(), scaled_sobelx.max())
binary_output_x = np.zeros_like(scaled_sobelx)
binary_output_x[(scaled_sobelx >= thresh_abs_x[0]) & (scaled_sobelx <= thresh_abs_x[1])] = 1
placeholder = np.dstack((placeholder, binary_output_x))

In [ ]:
# color space demo
from skimage import exposure

img = mpimg.imread('./test_images/vlcsnap-2017-08-02-08h34m19s558.jpg')
plt.figure(figsize=(17,5))
plt.imshow(img)
img = hist_equ(img)

plt.figure(figsize=(17,5))
plt.imshow(img)

img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
#img_hsv= img

plt.figure(figsize=(17,5))
plt.imshow(img_hsv[:,:,0], cmap='gray')
plt.figure(figsize=(17,5))
plt.imshow(img_hsv[:,:,1], cmap='gray')
plt.figure(figsize=(17,5))
plt.imshow(img_hsv[:,:,2], cmap='gray')

In [ ]:
# lane curvature
y_eval = 720



left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.abs(2*left_fit_cr[0])
right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.abs(2*right_fit_cr[0])
print(left_curverad, right_curverad, 'in meters')

In [ ]:
# test with custom sobel kernel
img = mpimg.imread('./test_images/circle.png')

kernel10 = np.array([[0, -3, -10],[3, 0, -3],[10, 3, 0]]) 

kernel01 = np.array([[10, 3, 0],[3, 0, -3],[0, -3, -10]]) 

gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

sobel_custom1 = cv2.filter2D(gray, -1, kernel10)
plt.figure(figsize=(17,10))
plt.imshow(sobel_custom1, cmap='gray')

sobel_custom2 = cv2.filter2D(gray, -1, kernel01)
plt.figure(figsize=(17,10))
plt.imshow(sobel_custom2, cmap='gray')

sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=-1)
plt.figure(figsize=(17,10))
plt.imshow(sobelx, cmap='gray')

sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=-1)
plt.figure(figsize=(17,10))
plt.imshow(sobely, cmap='gray')

In [ ]:
x = np.array(range(0,3))

for n in range(6,15):
    print(x.size)
    x = np.append(x, n)
    if x.size > 5:
        x = x[1:]
    print(x)



In [ ]:
x = [0,1,2]

for n in range(6,15):
    print(len(x))
    x.append(n)
    if len(x) > 5:
        x = x[1:]
    print(x)



In [ ]:
a = np.array([[50, 48, 4, 3, 2, 1, 1]])

In [ ]:
print(np.median(a), np.mean(a))

In [ ]:
a = np.array([])
a.ndim

In [ ]:
left_fit.ndim

In [ ]:
class _alt_Lines():
    def __init__(self):
        
        self.last_detected_lines = 0
        
        self.left_fit_pool = np.array([])
        self.curvature_left = 0
           
        self.right_fit_pool = []
        self.curvature_left = 0
        
        self.offset_lane = 0
        
    def process_img(self, img):
        if self.last_detected_lines == 0:
            left_fit, right_fit, left_fit_cr, right_fit_cr = find_lines(img)
            print(left_fit, right_fit, left_fit_cr, right_fit_cr)
        else:
            #TODO - simple line finding here
            print('bla')
        
        if self.sanitycheck_parameters(left_fit, right_fit, left_fit_cr, right_fit_cr):     
            self.update_parameters(left_fit, right_fit, left_fit_cr, right_fit_cr)
        
        
    def sanitycheck_parameters(self, left_fit, right_fit, left_fit_cr, right_fit_cr): 
        #TODO
        if 1 == 1:
            self.last_detected_lines += 1
            return True
        else:
            return False

    def update_parameters(self, left_fit, right_fit, left_fit_cr, right_fit_cr):
        
        
        if self.left_fit_pool.size == 0:
            self.left_fit_pool = left_fit
        else:
            self.left_fit_pool = np.vstack((self.left_fit_pool, left_fit))
            if self.left_fit_pool.shape[1] > 10:
                self.left_fit_pool = self.left_fit_pool[1:,:]
        print(np.median(left_fit_pool[:,0]))
        print(np.median(left_fit_pool[:,1]))
        print(np.median(left_fit_pool[:,2]))
         
        
        # Lane curvature  ##TODO: this has to be smoothed out as well. (np.median(self.left_fit_pool))
        y_eval = img.shape[0]
        self.curvature_left = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.abs(2*left_fit_cr[0])
        self.curvature_right = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.abs(2*right_fit_cr[0])
        print(self.curvature_left, self.curvature_right, 'in meters')
        
        ## Offset
        y= img.shape[0]
        xl = left_fit[0]* y**2 + left_fit[1]*y + left_fit[2]  
        xr = right_fit[0]* y**2 + right_fit[1]*y + right_fit[2]
        print('left: ', xl)
        print('right: ', xr)
        center_point = img.shape[1]/2
        self.offset_lane = (center_point - ((xr + xl)/2))*xm_per_pix
        print('Offset from middle of the lane: ', self.offset_lane*100, 'cm')

        
## TEST         
lines = Lines()
img = mpimg.imread('./test_images/test5.jpg')
red = get_redchannel(img)
thresh = apply_threshold_dynamic2(red, percentile=0.03)
birdview = birdview_warp(thresh)
lines.process_img(birdview)

In [ ]:
def find_lines(img):

    assert img.ndim == 2
    
    histogram = np.sum(img[360:,350:930], axis=0)
    midpoint = np.int(histogram.shape[0]/2)
    left_peak = np.argmax(histogram[:midpoint]) + 350
    right_peak = np.argmax(histogram[midpoint:]) + midpoint + 350
    print('left_peak: ', left_peak)
    print('right_peak: ', right_peak)
    
    nwindows = 9
    window_height = np.int(img.shape[0]/nwindows)
    #left_peak = 500
    #right_peak = 800
    margin = 100
    adjust_window_pos = 50
    left_lane_coordx = np.array([])
    left_lane_coordy = np.array([])
    right_lane_coordx = np.array([])
    right_lane_coordy = np.array([])

    for n in range(nwindows):
        window_lower_border = img.shape[0] - n*window_height
        window_slice = img[window_lower_border-window_height:window_lower_border, :]

        slice_left_window = window_slice[: , left_peak-margin:left_peak+margin]
        slice_left_line_coordy, slice_left_line_coordx = np.nonzero(slice_left_window)
        slice_left_line_coordy += window_lower_border - window_height
        slice_left_line_coordx += left_peak - 100

        left_lane_coordx = np.append(left_lane_coordx, slice_left_line_coordx)
        left_lane_coordy = np.append(left_lane_coordy, slice_left_line_coordy)

        if slice_left_line_coordx.size > adjust_window_pos:
            left_peak = np.int(np.mean(slice_left_line_coordx))


        slice_right_window = window_slice[: , right_peak-margin:right_peak+margin]
        slice_right_line_coordy, slice_right_line_coordx = np.nonzero(slice_right_window)
        slice_right_line_coordy += window_lower_border - window_height
        slice_right_line_coordx += right_peak - 100

        right_lane_coordx = np.append(right_lane_coordx, slice_right_line_coordx)
        right_lane_coordy = np.append(right_lane_coordy, slice_right_line_coordy)

        if slice_right_line_coordx.size > adjust_window_pos:
            right_peak = np.int(np.mean(slice_right_line_coordx))


    assert left_lane_coordx.size == left_lane_coordy.size
    assert right_lane_coordx.size == right_lane_coordy.size

    left_fit_cr = np.polyfit(left_lane_coordy*ym_per_pix, left_lane_coordx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(right_lane_coordy*ym_per_pix, right_lane_coordx*xm_per_pix, 2)    
    
    left_fit = np.polyfit(left_lane_coordy, left_lane_coordx, 2)
    right_fit = np.polyfit(right_lane_coordy, right_lane_coordx, 2)    
    

    return left_fit, right_fit, left_fit_cr, right_fit_cr





    
img = mpimg.imread('./test_images/test5.jpg')
red = get_redchannel(img)
thresh = apply_threshold_dynamic2(red, percentile=0.03)
birdview = birdview_warp(thresh)
left_fit, right_fit, left_fit_cr, right_fit_cr = find_lines(birdview)
print(left_fit, right_fit)

In [ ]:
def findlines(img):
    
    histogram = np.sum(img[int(img.shape[0]/2):,400:880], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((img, img, img))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(img.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base + 400
    rightx_current = rightx_base + 400
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    print(nonzeroy[left_lane_inds])
    
    
    plt.figure(figsize=(17,5))
    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.show()
    
    return left_fit, right_fit, leftx, lefty, rightx, righty




        
## Demo
img = mpimg.imread('./test_images/test4.jpg')
red = get_redchannel(img)
thresh = apply_threshold_dynamic2(red, percentile=0.03)
birdview = birdview_warp(thresh)
left_fit, right_fit, leftx, lefty, rightx, righty = findlines(birdview)
print_example_images(img, thresh)

print(leftx)
print(lefty)

# Generate x and y values for plotting
ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]


plt.figure(figsize=(17,5))
plt.imshow(birdview_warp(img), cmap='gray')
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)
plt.show()


In [ ]:
left_fit

In [ ]:
test = np.vstack((left_fit, left_fit))

In [ ]:
test

In [ ]:
test = np.vstack((test, right_fit))

In [ ]:
test

In [ ]:
test[1:,:]

In [ ]:
test = np.array([1,1,1])
for n in range(10):
    test = np.vstack((test, np.array([n,n,n])))
    
    if test.shape[0] > 3:
        test = test[1:,:]
    print(test)
    #print(test.mean())
    print(test.shape)